### Multiword Expressions


Wydzielaliśmy do tej pory słowa w izolacji, choć wiemy, że występują one grupami. Jak stwierdzić, czy to, co obserwujemy w tekście to przypadkowa konstrukcja (nazwy rzeczy wymieniane jednym tchem, występujące obok siebie takie a nie inne słowa) czy może stałe połączenie wyrazowe, które możemy badać jako zjawisko (na wiele sposobów!) lub wykluczyć z korpusu, który poddajemy badaniom. 

Trwałe połączenia wyrazowe to frazy, które znamy jako przysłowia, powiedzonka, nazwy, złożenia. Więcej o nich i ich istocie można poczytać tu: https://en.wikipedia.org/wiki/Multiword_expression. Warto jednak zaznaczyć, że koncepcja jednostki wielowyrazowej ulega modyfikacjom w zależności od obranego paradygmatu badawczego! Dobrze jest przemyśleć sobie, co rozumiemy przez tę konstrukcję i konsekwentnie stosować się do tej wizji w naszych badaniach.

Przedmiotem tego laboratorium jest koncept samego poszukiwania jednostek wielowyrazowych, ich opisu gramatycznego, zestawienia częstych z rzadkimi wystąpieniami w korpusie.

W ramach wykrywania bigramów, będziemy kolejno:
1. Pobierać i tokenizować korpus FIQA.
2. Obliczać liczbę wystąpień złożeń dwóch elementów w tekście. Dla znanego przez nas zdania: The quick brown fox jumped over lazy dog rozłożenie takich dwuelementowych segmentów wygląda następująco:
"the quick": 1
"quick brown": 1
"brown fox": 1
...
"dog .": 1
3. Wyselekcjonujemy te segmenty, które tworzone są tylko przez słowa (ciekawość badacza: dlaczego teraz? dlaczego działamy w taki sposób?)
4. Użyjemy miary Pointwise Mutual Informatio (PMI), by obliczyć siłę powiązań pomiędzy elementami tworzącymi segment. PMI (https://en.wikipedia.org/wiki/Pointwise_mutual_information) to tylko jeden ze sposobów na otrzymanie takiej informacji, inne opisano tu: https://onlinelibrary.wiley.com/doi/full/10.1111/lang.12225.
5. Znajdziemy najczęstsze segmenty występujące w tekście. Określimy też próg przypadkowych połączeń wyrazów jako 5 wystąpień, nie będzie nas interesowało nic, co występuje 5 razy i rzadziej.
6. Spojrzymy na cały problem z punktu widzenia składni, a więc oznaczymy segmenty, przypisując im interpretację morfologiczną, policzymy, które złożenia są najczęstsze (ciekawość badacza: jak najczęstrze kategorie morfologiczne mają się w porównaniu do naszych najczęstszych słów z wcześniejszych punktów?

Zaczynamy od przygotowania środowiska.

In [ ]:
import string
from collections import Counter, defaultdict
from itertools import chain

import numpy as np
import spacy
from datasets import load_dataset
from spacy.lang.pl import Polish

##!pip install ipywidgets

Użyte moduły:
- `string` https://docs.python.org/3/library/string.html
- `collections` https://docs.python.org/3/library/collections.html
- `itertools` https://docs.python.org/3/library/itertools.html

## Zadanie 1

Pobieramy korpus, tokenizujemy go, używając do tego znanych już narzędzi i zasobów.

In [ ]:
dataset = load_dataset('clarin-knext/fiqa-pl', 'corpus')['corpus']['text']
dataset = ' '.join(dataset)

nlp = Polish()
tokenizer = nlp.tokenizer

tokens = tokenizer(dataset)
tokens = list(map(lambda t: t.text.lower(), tokens))
tokens[:10]

## Zadanie 2-3

Obliczamy bigramy:
(tu sprytne wyjaśnienie: https://www.exploredatabase.com/2020/04/bigram-probability-estimation-of-word-sequence-example.html)

In [ ]:
bigrams = [a + ' ' + b for a, b in zip(tokens, tokens[1:])]
bigrams = list(filter(lambda x: len(x.split(' ')) == 2, bigrams))
bigrams_counter = Counter(bigrams)

for i, (key, val) in zip(range(1, 50), bigrams_counter.items()):
    print(f'{i}. {key}: {val}')

I filtrujemy bigramy, aby zachować tylko takie, które zawierają wyrazy składające się wyłącznie z liter polskiego alfabetu:

In [ ]:
def discard_elements(iterable, filter_fn):
    discard = []
    
    for element in iterable:
        if filter_fn(element):
            discard.append(element)
            continue
    
    for d in discard:
        del iterable[d]


all_chars = string.ascii_letters + 'ęóąśłżźćńĘÓĄŚŁŻŹĆŃ '
filter_not_letters = lambda element: any([c not in all_chars for c in element])

discard_elements(bigrams_counter, filter_not_letters)

for i, (key, val) in zip(range(1, 50), bigrams_counter.items()):
    print(f'{i}. {key}: {val}')

## Zadanie 4

Używamy PMI, żeby stwierdzić, jakie jest prawdopodobieństwo współwystępowania słów z bigramu w korpusie - PMI przybliża siłę powiązania słów. Im wyższy wynik, tym mocniej powiązane są słowa.

In [ ]:
tokens_counter = Counter(tokens)


def pmi(bigram):
    x, y = bigram.split(' ')
    p_x = tokens_counter[x] / len(tokens_counter)
    p_y = tokens_counter[y] / len(tokens_counter)
    p_xy = bigrams_counter[bigram] / len(bigrams_counter)

    if p_x * p_y == 0 or p_xy == 0:
        return 0
    else:
        return np.log2(p_xy / (p_x * p_y))


bigrams_pmi = [(pmi(bigram), bigram) for bigram in bigrams_counter]
bigrams_pmi[:50]

## Zadanie 5

Bezpośrednie zastosowanie PMI na korpusie zwraca dość charakterystyczne bigramy, które wyglądają, jakby występowały w korpusie bardzo rzadko (zarówno bigramy, jak i tworzące je tokeny) i dlatego znalazły się na początku rankingu (przez bardzo małe prawdopodobieństwa $p_x, p_y$).

In [ ]:
sorted(bigrams_pmi, reverse=True)[:50]

Odfiltrowanie najrzadszych bigramów pozwala uzyskać bardziej rozsądne wyniki - większość naszych aktualnych wyników to charakterystyczne określenia składające się z dwóch słów (np. "klęska żywiołowa", "książeczka czekowa") lub nazwy własne (np. "Stucco Veneziano", "Bert Hellinger").

In [ ]:
filter_rare = lambda element: bigrams_counter[element] < 5
discard_elements(bigrams_counter, filter_rare)

bigrams_pmi = [(pmi(bigram), bigram) for bigram in bigrams_counter]
top_bigrams_pmi = sorted(bigrams_pmi, reverse=True)[:50]
top_bigrams_pmi

## Zadanie 6

Na wyniki patrzymy teraz z innej perspektywy - chcemy zobaczyć, jak najczęstsze segmenty rozkładają się w ujęciu gramatycznym.

In [ ]:
nlp = spacy.load('pl_core_news_sm')

dataset = load_dataset('clarin-knext/fiqa-pl', 'corpus')['corpus']['text']
dataset = map(nlp, dataset)
dataset = list(map(lambda word: f'{word.lemma_}:{word.tag_}'.lower(), chain.from_iterable(dataset)))

for i, word in enumerate(dataset[:10], 1):
    print(f'{i}. {word}')

Ranking top-10 wartości PMI obliczony na korpusie po lematyzacji oraz tagowaniu daje bardzo podobne wyniiki - pojawiają się w nim te same lub należące do tych samych kategorii bigramy.

In [ ]:
bigrams = [a + ' ' + b for a, b in zip(dataset, dataset[1:])]
bigrams = list(filter(lambda x: len(x.split(' ')) == 2, bigrams))
bigrams_counter = Counter(bigrams)
tokens_counter = Counter(dataset)

def filter_not_letters(element):
    a, b = element.split(' ')
    a, b = a.split(':')[0], b.split(':')[0]
    return any([c not in all_chars for c in a + b])

discard_elements(bigrams_counter, filter_not_letters)
discard_elements(bigrams_counter, filter_rare)

for i, (key, val) in zip(range(1, 50), bigrams_counter.items()):
    print(f'{i}. {key}: {val}')

print()

bigrams_pmi = [(pmi(bigram), bigram) for bigram in bigrams_counter]
top_bigrams_pmi_tag_lem = sorted(bigrams_pmi, reverse=True)[:10]
top_bigrams_pmi_tag_lem

In [ ]:
groups = defaultdict(list)

for bigram, count in bigrams_counter.items():
    a, b = bigram.split(' ')
    a, b = a.split(':')[1], b.split(':')[1]
    groups[(a, b)].append((count, bigram))

for i, key in zip(range(1, 11), groups):
    print(f'{i}. {key}')

## Podsumowanie

Kategorie najczęściej spotykanych bigramów są dość intuicyjne, jeśli chodzi o język naturalny, wyniki są zgodne z regułami łączenia słów w języku polskim:

- przyimek + rzeczownik (np. "na przykład"),
- przymiotnik + rzeczownik (np. "druga strona"),
- rzeczownik + przyimek (np. "podatek od"),
- rzeczownik + rzeczownik (np. "miejsce pracy"),
- przyimek + przymiotnik (np. "w którym"),
- rzeczownik + przymiotnik (np. "karta kredytowa"),
- rzeczownik + czasownik (np. "to jest"),
- przeczenie + czasownik (np. "nie jest"),
- czasownik + rzeczownik (np. "oznacza to"),
- rzeczownik + spójnik (np. "pieniądz i").

In [ ]:
groups_count = [(sum(map(lambda g: g[0], val)), key) for key, val in groups.items()]
groups_count = sorted(groups_count, reverse=True)
groups_count[:10]

In [ ]:
for _, group in groups_count[:10]:
    print('-' * 50)
    print(group)
    
    for i, v in enumerate(sorted(groups[group], reverse=True)[:5], 1):
        print(f'{i}. [{v[0]}] {v[1]}')
        
print('-' * 50)